In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from PIL import Image
import requests
from sklearn.preprocessing import OneHotEncoder
import os
import cv2 as cv
import rasterio
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def norm(band):
  band_min, band_max = band.min(), band.max()
  return ((band - band_min)/(band_max - band_min))

def get_image(image_path):
  src = rasterio.open(image_path, 'r')
  S_images=src.read()
  c0 = norm(S_images[0].astype(np.float32))
  c1 = norm(S_images[1].astype(np.float32))
  c2 = norm(S_images[2].astype(np.float32))
  c3 = norm(S_images[3].astype(np.float32))
  c4 = norm(S_images[4].astype(np.float32))
  c5 = norm(S_images[5].astype(np.float32))
  image = np.dstack((c0,c1,c2,c3,c4,c5))
  src.close()
  return image

def process_image(satellite_image):
  h,w,c = satellite_image.shape
  if w < h:
    satellite_image = np.transpose(satellite_image, (1, 0, 2))
  resized_image = cv.resize(satellite_image, (25, 25), interpolation=cv.INTER_CUBIC)
  return resized_image

def get_age(year, location, time_point, planting_date):
  collection_date = doc[str(year)][location][time_point]
  planting_date = pd.to_datetime(planting_date)
  collection_date = pd.to_datetime(collection_date)
  return (collection_date-planting_date).days

In [4]:
val_2023_split = pd.read_csv('data_processed/val_2023.csv')

In [5]:
f = open('data_processed/doc_2023.pkl', 'rb')
doc_2023 = pickle.load(f)
f.close()
f = open('data_processed/val_2023.pkl', 'rb')
data_split = pickle.load(f)
f.close()

In [6]:
doc = {}
doc['2023'] = doc_2023

Steps in data preprocessing:


*   Remove Scottsbluff from training set (scottsbluff plot length is different from all other data)
*   Remove rows with nan
*   Remove genotype (2023 val set has two genotypes that are new) and irrigationProvided
*   Divide yield by max yield. Store max yield in metadata
*   Convert location, N_level into one hot encoded form
*   Get the satellite images for each row. Transpose image if width is shorter than height. Reside to 25 by 25. And min max normalize each channel of the image.
*   For each satellite image, Calculate age by finding difference between planting date and time point. Divide age by 365.
*   Make a list of data samples and another list of yields
*   Divide the list into training vs validation
*   Save as numpy array or pickle file



In [8]:
all_data = val_2023_split

In [9]:
all_data.head()

,year,location,irrigationProvided,experiment,range,row,N_level,genotype,planting_date,location_Ames,location_Crawfordsville,location_Lincoln,location_MOValley,N_level_High,N_level_Low,N_level_Medium
0,2023,Ames,0,2315,12,11,Low,PHN46 X PHB47,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2023,Ames,0,2315,3,13,Low,B73 X MO17,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2023,Ames,0,2315,18,13,Low,B73 X PHM49,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2023,Ames,0,2315,2,15,Low,LH185 X LH82,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023,Ames,0,2315,5,15,Low,2369 X PHN82,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
with open('processed_data.pkl', 'rb') as f:
  processed_data = pickle.load(f)

In [11]:
max_yield = processed_data['metadata']['max_yield']

In [12]:
all_data = all_data.drop(columns=['genotype','irrigationProvided'])

In [13]:
all_data.head()

,year,location,experiment,range,row,N_level,planting_date,location_Ames,location_Crawfordsville,location_Lincoln,location_MOValley,N_level_High,N_level_Low,N_level_Medium
0,2023,Ames,2315,12,11,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2023,Ames,2315,3,13,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2023,Ames,2315,18,13,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2023,Ames,2315,2,15,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023,Ames,2315,5,15,Low,5/19/23,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
images = []
numerical_features = []

for index, row in tqdm(all_data.iterrows()):

  location = row['location']
  year = row['year']
  if year == 2023:
    year = '2023_val'
  experiment = row['experiment']
  if experiment == 'Hyrbrids':
    experiment = 'Hybrids'
  range = row['range']
  row_ = row['row']
  planting_date = row['planting_date']
  time_points = sorted(os.listdir(os.path.join('satellite_images', str(year), location)))

  new_images = []
  new_numerical_features = []  
  for time_point in time_points:
    folder_path = os.path.join('satellite_images', str(year), location, time_point)
    image_name = f'{location}-{time_point}-{experiment}_{range}_{row_}.TIF'
    image_path = os.path.join(folder_path, image_name)
    if os.path.exists(image_path):
      satellite_image = get_image(image_path)
      processed_image = process_image(satellite_image)
      age = get_age(row['year'], location, time_point, planting_date)
      age = age/365
      new_images.append(processed_image)
      new_numerical_features.append([row['location_Ames'],
                     row['location_Crawfordsville'],
                     row['location_Lincoln'],
                     row['location_MOValley'],
                     row['N_level_High'],
                     row['N_level_Low'],
                     row['N_level_Medium'],
                     age])
      print('found ',image_path)  
    else:
      print('not_found', image_path)
  images.append(new_images)
  numerical_features.append(new_numerical_features)  

images = np.array(images)
numerical_features = np.array(numerical_features)

4it [00:00, 29.74it/s]

found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_12_11.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_12_11.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_12_11.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_3_13.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_3_13.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_3_13.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_18_13.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_18_13.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_18_13.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_2_15.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_2_15.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_2_15.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_5_15.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_5_15.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_5_15.TIF
found  satellite_im

12it [00:00, 35.71it/s]

found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_6_11.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_6_15.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_6_15.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_6_15.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_22_11.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_22_11.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_22_11.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_5_17.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_5_17.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_5_17.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_13_13.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_13_13.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2315_13_13.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2315_21_15.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2315_21_15.TIF
found  satellite_

20it [00:00, 36.77it/s]

...

found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_30_59.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_30_59.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_30_59.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_33_61.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_33_61.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_33_61.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_30_57.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_30_57.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_30_57.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_43_59.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_43_59.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_43_59.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_36_55.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_36_55.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_36_55.TIF
found  sat

275it [00:08, 29.35it/s]

found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_43_55.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_43_55.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_43_55.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_32_59.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_32_59.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_32_59.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_44_61.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_44_61.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_44_61.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_39_55.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_39_55.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_39_55.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_42_57.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_42_57.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_42_57.TIF
found  sat

282it [00:08, 31.46it/s]

found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_42_59.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_42_59.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_42_59.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_36_61.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_36_61.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_36_61.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_32_61.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_32_61.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_32_61.TIF
found  satellite_images\2023_val\Ames\TP1\Ames-TP1-2335_45_59.TIF
found  satellite_images\2023_val\Ames\TP2\Ames-TP2-2335_45_59.TIF
found  satellite_images\2023_val\Ames\TP3\Ames-TP3-2335_45_59.TIF


In [16]:
data_dict = {'images': images,
             'numerical_features': numerical_features,
             'metadata': processed_data['metadata']}

In [18]:
f = open('processed_val_data_2023.pkl', 'wb')
pickle.dump(data_dict, f)
f.close()